In [1]:
#include <concepts>
#include <iostream>
#include <vector>

## 3.1 concept

In [2]:
template<typename T> 
concept is_integral = std::is_integral_v<T>;

In [3]:
std::cout << is_integral<int> << std::endl;

1


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f9b1e417500


In [4]:
std::cout << is_integral<double> << std::endl;

0


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f9b1e417500


In [5]:
template<typename T> 
concept is_floating_point = std::is_floating_point_v<T>;

In [6]:
template<typename T>
concept is_real = is_integral<T> || is_floating_point<T>;

In [7]:
is_real<decltype(0.0)>

(bool) true


In [8]:
is_real<decltype(1)> 

(bool) true


In [9]:
template<typename T>
concept is_not_real = ! is_real<T>;

In [10]:
is_not_real<decltype(1)>

(bool) false


In [11]:
is_not_real<decltype(std::true_type{})>

(bool) true


## 3.2 requires

In [12]:
template<typename M>
concept Machine = requires(M m){
    m.powerUp();
    m.powerDown();
};

In [13]:
struct motor{
    void powerUp(){};
    void powerDown(){};
};

In [14]:
Machine<motor>;

(bool) true


In [15]:
struct Toy{
    double powerUp(){return 1.0;};
    double powerDown(){return 0;};
};

In [16]:
Machine<Toy>;

(bool) true


In [17]:
template<typename T>
concept C = requires{
    typename T::type;
    typename std::vector<T>;
};

In [18]:
struct Foo{ using type = float; };

In [19]:
C<Foo>;

(bool) true


In [20]:
sizeof(Foo);

(unsigned long) 1


In [21]:
template<typename T>
concept Movable = requires(T a, T b){
    { a = std::move(b) } noexcept ;
};

In [22]:
Movable<std::vector<float>>

(bool) true


In [23]:
template<typename T> 
concept C1  = requires(T t){
    {t.f()} -> std::same_as<int>;
    {t.g()} -> std::convertible_to<double>;
};

In [24]:
struct Bar{
    int f(){return 1;};
    double g(){return 0.0;};
};

In [25]:
C1<Bar>

(bool) true


In [26]:
template<typename T>
concept C2 = requires(T t){
    t.f();
    requires std::same_as<decltype((t.f())),int>;
    t.g();
    requires std::convertible_to<decltype((t.g())), double>;
};

In [27]:
C2<Bar> 

(bool) true


In [28]:
template<typename T> 
constexpr bool has_member_swap = requires(T a, T b){
    a.swap(b);
};

In [29]:
has_member_swap<std::vector<float>>;

(const bool) true


In [30]:
has_member_swap<int>;

(const bool) false


We get true or compilation time error when using `requires`.

In [31]:
// constexpr bool has_int_member_swap = requires(int a, int b){
//     a.swap(b);
// };

Below is a wrong case:

In [32]:
template<size_t N> 
constexpr bool IsEven = requires{
    N % 2 == 0;
};

In [33]:
IsEven<2>

(const bool) true


In [34]:
IsEven<1>

(const bool) true


Below is a right case:

In [35]:
template<size_t N> 
constexpr bool Even = requires{
    requires (N % 2 == 0);
};

In [36]:
Even<2>

(const bool) true


In [37]:
Even<1>

(const bool) false


Some mechanism like `SFINAE` is used here when requires leads to compilation time error.

In [38]:
template <typename T> 
void clever_swap(T& a, T& b){
    if constexpr (requires(T a, T b){a.swap(b);}){
        a.swap(b);
    }else{
        std::swap(a, b);
    }
};

In [39]:
std::vector v1{1}, v2{2};

In [40]:
clever_swap(v1, v2);

(void) @0x7f9b027fb030


In [41]:
v1

(std::vector<int, allocator<int> > &) { 2 }


In [42]:
v2

(std::vector<int, allocator<int> > &) { 1 }


In [43]:
int i1 = 1, i2 = 2;

In [44]:
clever_swap(i1, i2);

(void) @0x7f9b027fb030


In [45]:
i1

(int) 2


In [46]:
i2

(int) 1


In [47]:
// template<typename T>
// constexpr bool self_plus = requires (T v){
//     requires (typename T::value_type x){++x;};
// };

`(typename T::value_type x){++x;}` is not a valid bool expression.

In [48]:
template<typename T>
constexpr bool self_plus =  requires (T v){
    requires requires (typename T::value_type x){++x;};
};

GCC accept the code above and `self_plus<int>` below is false, we do not get a compilation time error.

In [49]:
self_plus<int>

(const bool) false


In [50]:
struct Plus{
    using value_type = int;
};

In [51]:
self_plus<Plus>

(const bool) true


## 3.3 requires clause

In [52]:
template<typename T> 
requires std::is_integral_v<T>
T simple_add(T a, T b){
    return a + b;
}

In [53]:
// simple_add(1.0, 2.0);

In [54]:
simple_add(1, 2);

(int) 3


In [55]:
template<typename T>
void f(T) {std::cout << "2" << std::endl;}

template<typename T>
requires std::is_trivial_v<T>
void f(T) {std::cout << "1" << std::endl;}

In [56]:
f("haha");

1


(void) @0x7f9b027fb030


In [57]:
f(std::vector<int>{});

2


(void) @0x7f9b027fb030


In [58]:
f(std::vector<double>{});

2


(void) @0x7f9b027fb030


In [59]:
template<typename T>
void PushSomeone(T& someone){
    std::cout << "Just do nothing and wait for end of workday." << std::endl;
}

template<typename T>
requires requires(T worker){ worker.DoSomeWork();}
void PushSomeone(T& someone){
    someone.DoSomeWork();
}

In [60]:
struct Worker{
    void DoSomeWork(){
        std::cout << "Write some code." << std::endl;
    }
};

struct LazyWorker{
    
};

In [61]:
Worker worker;
LazyWorker lazy_worker;

In [62]:
PushSomeone(worker);

Write some code.


(void) @0x7f9b027fb030


In [63]:
PushSomeone(lazy_worker);

Just do nothing and wait for end of workday.


(void) @0x7f9b027fb030


In [64]:
template<std::integral T, std::integral U>
typename std::conditional_t<sizeof(T) >= sizeof(U), T, U> integral_plus(T t, U u){
    return t + u;
}

In [65]:
integral_plus(1, 2);

(int) 3


In [66]:
integral_plus(1u, 2u);

(unsigned int) 3


In [67]:
uint8_t u1 = 1;
uint64_t u2 = 2;
integral_plus(u1, u2);

(unsigned long) 3


see generic lambda with concept:

In [68]:
auto integral_plus_lambda = [](std::integral auto a, std::integral auto b){
    return a + b;
};

In [69]:
// integral_plus_lambda(1, 2);

code above does not compile and it compiles ok on Compiler Explorer:
```C++
#include <concepts>
#include <iostream> 


auto integral_plus_lambda = [](std::integral auto a, std::integral auto b){
    return a + b;
};


int main(){
    std::cout << integral_plus_lambda(1, 2) << std::endl;
}
```

For more specific member function generation in template, requires overperform std::enable_if_t:

In [70]:
template <typename T>
struct Wrapper{
    T value_;
    void operator()() requires std::is_invocable_v<T> {
        value_();
    }
    void reset(T v) {value_ = v;}
};

In [71]:
Wrapper<int> wi;

In [72]:
std::cout << wi.value_ << std::endl;
wi.reset(1);
std::cout << wi.value_ << std::endl;

0
1


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f9b1e417500


In [73]:
// wi();

In [74]:
struct SaySomething{
    void operator()(){
        std::cout << "Something" << std::endl;
    }
};

In [75]:
Wrapper<SaySomething> ws;

In [76]:
// ws();

In [77]:
// std::is_invocable_v<SaySomething> 

There is something wrong with the 2 above cells and the code below compiles in Compiler Explorer:
```C++
#include <concepts>
#include <iostream> 

template <typename T>
struct Wrapper{
    T value_;
    void operator()() requires std::is_invocable_v<T> {
        value_();
    }
    void reset(T v) {value_ = v;}
};

struct SaySomething{
    void operator()(){
        std::cout << "Something" << std::endl;
    }
};

int main(){
    std::cout << std::is_invocable_v<SaySomething>  << std::endl;
    Wrapper<SaySomething> ws;
    ws();
}
```

## 3.4 partial ordering rules for concept constraint